In [ ]:
import autograd.numpy as np 
from autograd import grad

# Algorithm 1: Dual Averaging with D-Adaptation
def dual_avg_dadapt(func, x0, d0, n=100):
  assert d0 > 0
  dfdx = grad(func)

  gamma = np.zeros(n+1)
  g = np.zeros(n)
  s = np.zeros(n+1)
  d = np.zeros(n+1)
  d_hat = np.zeros(n+1)
  x = np.zeros(n+1)
  
  gamma[0] = 1/np.abs(dfdx(x0))
  s[0] = 0
  d[0] = d0
  d_hat[0] = d0
  x[0] = x0

  if dfdx(x[0]) == 0:
    return x[0]

  for k in range(n):
    g[k] = dfdx(x[k])
    s[k+1] = s[k] + d[k]*g[k]
    gamma[k+1] = 1/np.sqrt(np.sum(np.square(g)))
    d_hat[k+1] = (gamma[k+1] * np.square(s[k+1]) - np.sum(gamma[:k]*np.square(d[:k])*np.square(g[:k])))/(2*np.abs(s[k+1]))
    d[k+1] = np.max((d_hat[k+1], d[k]))
    x[k+1] = x0 - gamma[k+1]*s[k+1]
  
  x_hatn = np.sum(d[:n]*x[:n])/np.sum(d[:n])
  return x_hatn, x, d, d_hat

In [ ]:
# plot figure 1
import matplotlib
import matplotlib.pyplot as plt

def abs(x):
    return np.abs(x)

x_hatn, x, d, d_hat = dual_avg_dadapt(abs, 1.0, 1e-2)

matplotlib.rc('font', size=15)

fig, axs = plt.subplots(1, 2, figsize=(9, 4))
plt.sca(axs[0])
plt.plot(np.linspace(-1, 1, 1000), abs(np.linspace(-1, 1, 1000)), color='k')
plt.scatter(x, abs(x), color='#083D77')
plt.xlabel(r'$x$')
plt.ylabel(r'$f(x)$')

plt.sca(axs[1])
plt.plot(d, color='#083D77')
plt.plot(d_hat, color='#F78764')
plt.axhline(1, color='k', linestyle='--')
plt.xlabel('Step')
plt.ylabel('d')
plt.legend(['$d_k$', r'$\hat{d}_k$', '$D$'], frameon=False, loc='upper right')

plt.tight_layout()
plt.savefig('figure1.png')
plt.show()